<a href="https://colab.research.google.com/github/aCStandke/ReinforcementLearning/blob/main/SpyTradingAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Trading Using Reinforcment Learning


This part implementation comes from chapter 8 from the  book 
[Deep Reinforcement Learning Hands-On - Second Edition by Maxim Lapan](https://www.amazon.com/Deep-Reinforcement-Learning-Hands-optimization/dp/1838826998/ref=asc_df_1838826998/?tag=hyprod-20&linkCode=df0&hvadid=416741343328&hvpos=&hvnetw=g&hvrand=7234438034400691228&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9008183&hvtargid=pla-871456510229&psc=1&tag=&ref=&adgrpid=93867144477&hvpone=&hvptwo=&hvadid=416741343328&hvpos=&hvnetw=g&hvrand=7234438034400691228&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9008183&hvtargid=pla-871456510229)

 
Namely, a RL agent has some observation of the market, and has to take an action to either buy, sell, or hold. If the agent buys before the price goes up, profit will be positive; otherwise, the agent will get a negative reward. The agent is tyring to obtain as much profit as possible in the trading environment. 



In [ ]:
!pip install ptan

In [ ]:
!pip install tensorboardX

In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as nn_utils
import torch.multiprocessing as mp
from multiprocessing import cpu_count
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import glob
import os
import collections
import csv
import sys
import time
import ptan
import torch.optim as optim
from tensorboardX import SummaryWriter
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt

# Checking to see how many parallel processes can be spawned

In [ ]:
cpu_count()

# Validation Environment

In [ ]:
# env for validation (i.e. future market data for the agent to trade on)
def validation_run(env, net, episodes=100, device="cpu", epsilon=0.02, comission=0.1):
    stats = {
        'episode_reward': [],
        'episode_steps': [],
        'order_profits': [],
        'order_steps': [],
    }

    for episode in range(episodes):
        obs = env.reset()

        total_reward = 0.0
        position = None
        position_steps = None
        episode_steps = 0

        while True:
            obs_v = torch.tensor([obs]).to(device)
            out_v = net(obs_v)

            action_idx = out_v.max(dim=1)[1].item()
            if np.random.random() < epsilon:
                action_idx = env.action_space.sample()
            action = Actions(action_idx)

            close_price = env._state._cur_close()

            if action == Actions.Buy and position is None:
                position = close_price
                position_steps = 0
            elif action == Actions.Close and position is not None:
                profit = close_price - position - (close_price + position) * comission / 100
                profit = 100.0 * profit / position
                stats['order_profits'].append(profit)
                stats['order_steps'].append(position_steps)
                position = None
                position_steps = None

            obs, reward, done, _ = env.step(action_idx)
            total_reward += reward
            episode_steps += 1
            if position_steps is not None:
                position_steps += 1
            if done:
                if position is not None:
                    profit = close_price - position - (close_price + position) * comission / 100
                    profit = 100.0 * profit / position
                    stats['order_profits'].append(profit)
                    stats['order_steps'].append(position_steps)
                break

        stats['episode_reward'].append(total_reward)
        stats['episode_steps'].append(episode_steps)

    return { key: np.mean(vals) for key, vals in stats.items() }

# Loaing Data for Trading Environment


In [ ]:
Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])


def read_csv(file_name, sep=',', filter_data=True, fix_open_prices=False):
  print("Reading", file_name)
  with open(file_name, 'r') as fd:
    reader = csv.reader(fd)
    h = next(reader)
    if 'Open' not in h and sep == ',':
      return read_csv(file_name, ';')
    indices = [h.index(s) for s in ('Open', 'High', 'Low', 'Close', 'Volume')]
    #indices = [h.index(s) for s in ('<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>')]
    o, h, l, c, v = [], [], [], [], []
    count_out = 0
    count_filter = 0 
    count_fixed = 0
    prev_vals = None
    for row in reader:
      vals = list(map(float, [row[idx] for idx in indices])) 
      if filter_data and all(map(lambda v: abs(v-vals[0]) < 1e-8, vals[:-1])):
        count_filter += 1
        continue
      
      po, ph, pl, pc, pv = vals
      
      # putting price data into list and then into a np.array 
      # where o is open price, c is close price, h is high price, l 
      # is low price, and v is volume
      count_out +=1
      o.append(po)
      c.append(pc)
      h.append(ph)
      l.append(pl)
      v.append(pv)
      prev_vals = vals
  return Prices(open=np.array(o, dtype=np.float32),high=np.array(h, dtype=np.float32), low=np.array(l, dtype=np.float32),close=np.array(c, dtype=np.float32), volume=np.array(v, dtype=np.float32))


# prices(object): of collections.namedtuple type
def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)

def load_relative(csv_file):
    return prices_to_relative(read_csv(csv_file))


def price_files(dir_name):
    result = []
    for path in glob.glob(os.path.join(dir_name, "*.csv")):
        result.append(path)
    return result


def load_year_data(year, basedir='data'):
    y = str(year)[-2:]
    result = {}
    for path in glob.glob(os.path.join(basedir, "*_%s*.csv" % y)):
        result[path] = load_relative(path)
    return result


# Creating Trading Environment using gym.Env 





In [ ]:
# defualt number of past trading days agent can observe when taking action;
# for 1D convolution model, this is the column portion of 2D matrix
# default is 5 days
BARS_COUNT = 5
# default percentage of stock price trading agent pays broker when 
# buying/selling, default is 0.1% (i.e. very reasonable)
DEFAULT_COMMISSION_PERC = 0.1


# Actions
class Actions(enum.Enum):
  # actions agent can take when trading
  Hold = 0 
  Buy = 1
  Close = 2

# StocksEnv
class StocksEnv(gym.Env):

  # fields required by gym.Env
  metadata = {'render.modes': ['human']}
  spec = EnvSpec("SPYEnv-v0")

  def __init__(self, prices, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=True, state_1d=False, random_ofs_on_reset=True,
               reward_on_close=False, volumes=True):
    assert isinstance(prices, dict)
    self._prices = prices

#---------------------State-Observation Encoding Section------------------------    
    # key!!!: creating the state observation for trading agent; when using
    # the 1D convolutional model, encoding must be of State1D class!!!!
    if state_1d:
      self._state = State1D(bars_count, commission, reset_on_close, reward_on_close=reward_on_close, volumes=volumes)
    else:
      self._state = State(bars_count, commission, reset_on_close, reward_on_close=reward_on_close, volumes=volumes)
    
    # creating action space for trading agent
    self.action_space = gym.spaces.Discrete(n=len(Actions))
    
    # creating observation space for training agent
    self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=self._state.shape, dtype=np.float32)
    
    # decide if want to use random offset, default is True
    self.random_ofs_on_reset = random_ofs_on_reset
    self.seed()

#------------------------Reset Section------------------------------------------
  # creates the offset for time series data (i.e. not 
  # always starting at the beggining of the time series data for episode)
  def reset(self):
    self._instrument = self.np_random.choice(list(self._prices.keys()))
    prices = self._prices[self._instrument]
    bars = self._state.bars_count
    if self.random_ofs_on_reset:
      offset = self.np_random.choice(prices.high.shape[0]-bars*10)+bars
    else:
      offset = bars
    self._state.reset(prices, offset)
    return self._state.encode()  

#-----------------------Step Section--------------------------------------------
  # executes the sequence of agent taking action, getting reward and
  # then getting the next observation/state 
  def step(self, action_idx):
    action = Actions(action_idx)
    reward, done = self._state.step(action)
    obs = self._state.encode()
    info = {"instrument":self._instrument, "offset": self._state._offset}
    # self._state.render(reward)
    return obs, reward, done, info

#----------------------Render Section-------------------------------------------
  # required by gym.Env object; future will implement the render method to view
  # the observation space of agent when trading to compare analysis
  def render(self, mode='human', close=False):
    pass  
  def close(self):
    pass

  def seed(self, seed=None):
    self.np_random, seed1 = seeding.np_random(seed)
    seed2 = seeding.hash_seed(seed1+1) % 2**33
    return [seed1, seed2]

#----------------------Class Method Section-------------------------------------
  # creates the instance of the  StocksEnv object (can be used with differnt
  # stock data sets for multiple environment)
  @classmethod
  def from_dir(cls, data_dir, **kwargs):
    prices = {f: load_relative(f) for f in price_files(data_dir)}
    return StocksEnv(prices, **kwargs)

# State Space

The reward is of ***either/or form***. This ***either/or form*** of the reward is done by setting the variable reward_on_close to either True or False, which 
is a Boolean parameter that switches between the two reward schemes. If it is set to True, the agent will receive a reward only on the close/selling of its stock position. If set to false, the agent will recive a reward only the buying and holding of its stock position (i.e. not selling).The default setting is True for reward_on_close which amounts to the trading strategy of [active investing](https://www.investopedia.com/terms/a/activeinvesting.asp#:~:text=Active%20investing%20refers%20to%20an,activity%20to%20exploit%20profitable%20conditions.). And Changing reward_on_close to False amounts to the trading strategy of [passive investing](https://www.investopedia.com/terms/p/passiveinvesting.asp#:~:text=Passive%20investing's%20goal%20is%20to,price%20fluctuations%20or%20market%20timing.)


In [ ]:
# General State Class (i.e. models not based on convolutions)
class State:
  def __init__(self, bars_count, commission_perc, reset_on_close, reward_on_close=True, volumes=True):
    super(State, self).__init__()
    # checking bars_count is an int
    assert isinstance(bars_count, int)
    # checking that bars_count is greater than zero
    assert bars_count > 0
    # checking commission is a float
    assert isinstance(commission_perc, float)
    # checking commission is greater than zero
    assert commission_perc >= 0.0
    # checking that reset_on_close and reward on close are bools
    assert isinstance(reset_on_close, bool)
    assert isinstance(reward_on_close, bool)
    self.bars_count=bars_count
    self.commission_perc = commission_perc
    self.reset_on_close = reset_on_close
    self.reward_on_close = reward_on_close
    self.volumes = volumes
    self.bought_price = 0.0
  
  # method that reset's the environment 
  def reset(self, prices, offset):
    assert isinstance(prices, Prices)
    assert offset >= self.bars_count-1
    self.have_position = False  # start with no stocks
    self.open_price = 0.0
    self._prices = prices
    self._offset = offset

  # shape of observation space is 1D
  @property
  def shape(self):
    # the shape is the high, low, and closing prices of the current trading day
    # (i.e. 3 or 4 if volume is used) times the num of bars
    # (i.e. past prices agent can observe) plus the position flag 
    # (i.e. whether agent is holding onto the stock or not) and 
    # the relative profit agent has recieved since opening
    if self.volumes:
      return (5*self.bars_count+1+1, )
    else:
      return (4*self.bars_count+1+1, )
  
  # encoding observation space as a 1d Vector
  def encode(self):
    res = np.ndarray(shape=self.shape, dtype=np.float32)
    shift = 0
    for bar_idx in range(-self.bars_count+1, 1):
      res[shift] = self._prices.open[self._offset + bar_idx]
      shift += 1
      res[shift] = self._prices.high[self._offset + bar_idx]
      shift += 1
      res[shift] = self._prices.low[self._offset + bar_idx]
      shift += 1
      res[shift] = self._prices.close[self._offset + bar_idx]
      shift += 1
      if self.volumes:
        res[shift] = self._prices.volume[self._offset + bar_idx]
        shift += 1
    res[shift] = float(self.have_position)
    shift += 1
    if not self.have_position:
      res[shift] = 0.0
    else:
      res[shift] = (self._cur_close() - self.open_price) / self.open_price
    return res
 
  def _cur_close(self):
    """
    Calculate real close price for the current bar
    """
    open = self._prices.open[self._offset]
    rel_close = self._prices.close[self._offset]
    return open * (1.0 + rel_close)

#---------------!!!Step Section & Reward Calculation!!!------------------------- 
  def step(self, action):
      """
      Perform one step in our price, adjust offset, check for the end of prices
      and handle position change
      :param action:
      :return: reward, done
      """
      assert isinstance(action, Actions)
      reward = 0.0
      done = False
      close = self._cur_close()

      if action == Actions.Buy and not self.have_position:
        self.have_position = True
        self.open_price = close
        self.bought_price = close
        reward -= self.commission_perc
      elif action == Actions.Close and self.have_position:
        reward -= self.commission_perc
        done |= self.reset_on_close
        """
        implements the active investing strategy,since a reward is only given 
        when selling;a positive reward is given when selling higher than when 
        you bought and a negative reward is given when selling lower than when 
        you bought the stock 
        """
        if self.reward_on_close:
          reward += 100.0 * (close - self.open_price) / self.open_price
        self.have_position = False
        self.open_price = 0.0

      self._offset += 1
      prev_close = close
      close = self._cur_close()
      done |= self._offset >= self._prices.close.shape[0]-1

      """
      implements the passive investing strategy ,since you only get this reward 
      when not selling; you get a positive reward when the current closing price 
      is higher than the price you bought the stock and you get a negative 
      reward when the current closing price is lower than the price you bought 
      the stock   
      """
      if self.have_position and not self.reward_on_close:
        reward += 100.0 * (close - self.bought_price) / self.bought_price

      return reward, done
  
  # !!!!!TODO!!!rendering agent's observation space to screen!!!!!!!!!!!!!!!!!!
  def render(self, reward, mode='human', close=False):
    # creating plot object
    # plt.clf()
    # ofs = self.bars_count-1
    # plt.plot(self._prices.open[self._offset-ofs:self._offset+1])
    # plt.title("opening prices, data=%s" % "SPY ETF")
    # plt.ylabel("prices, %")
    # plt.show()
    pass

  def close(self):
    pass

# Specific State Class for encoding observation space for convolution models
class State1D(State):
    """
    State with shape suitable for 1D convolution, must be 2D of form 
    (row, column) where row is either 6 for just price data and vol or 
    7 depending on position
    """

    # shape of observation space is 2D
    @property
    def shape(self):
        if self.volumes:
            return (7, self.bars_count)
        else:
            return (6, self.bars_count)

    # encoding observation space as a 2d Matrix
    def encode(self):
        res = np.zeros(shape=self.shape, dtype=np.float32)
        ofs = self.bars_count-1
        res[0] = self._prices.open[self._offset-ofs:self._offset+1]
        res[1] = self._prices.high[self._offset-ofs:self._offset+1]
        res[2] = self._prices.low[self._offset-ofs:self._offset+1]
        res[3] = self._prices.close[self._offset-ofs:self._offset+1]
        if self.volumes:
            res[4] = self._prices.volume[self._offset-ofs:self._offset+1]
            dst = 5
        else:
            dst = 4
        if self.have_position:
            res[dst] = 1.0
            res[dst+1] = (self._cur_close() - self.open_price) / self.open_price
        return res

# Creating the Dueling DQN Network using 1D Convolutions 


In [ ]:
class DQNConv1D(nn.Module):
    def __init__(self, shape, actions_n, kernel):
        super(DQNConv1D, self).__init__()
        
        # 1D Convolutions for first transformation (i.e. cross-correlation)
        self.conv = nn.Sequential(
            nn.Conv1d(shape[0], 128, kernel),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel),
            nn.ReLU(),
        )

        out_size = self._get_conv_out(shape)

        # linear transformation for feed-forward value 
        self.fc_val = nn.Sequential(
            nn.Linear(out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

        # linear transformation for feed-forward advantage
        self.fc_adv = nn.Sequential(
            nn.Linear(out_size, 512),
            nn.ReLU(),
            nn.Linear(512, actions_n)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        val = self.fc_val(conv_out)
        adv = self.fc_adv(conv_out)
        return val + adv - adv.mean(dim=1, keepdim=True)



# Creating the Policy Gradient Network using 1D Convolutions

In [ ]:
class PGN(nn.Module):
    def __init__(self, shape, n_actions, kernel):
        super(PGN, self).__init__()
        
        # 1D Convolutions for first transformation (i.e. cross-correlation)
        self.conv_1 = nn.Sequential(
            nn.Conv1d(shape[0], 128, kernel),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel),
            nn.ReLU()
        )
        
        # linear transformation for policy net
        self.policy = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )


    def forward(self, x):
        output = self.conv_1(x)
        output = output.reshape(output.size(0), -1)
        output = self.policy(output)
        return output

# Creating A2C Network using 1D Convolutions 

In [ ]:
class A2C(nn.Module):
    def __init__(self, shape, actions_n, kernel):
        super(A2C, self).__init__()

        # 1D Convolutions for first transformation (i.e. cross-correlation)
        self.conv = nn.Sequential(
            nn.Conv1d(shape[0], 128, kernel),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel),
            nn.ReLU(),
        )
        
        conv_out_size = self._get_conv_out(shape)
        
        self.policy = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, actions_n)
        )

        self.value = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.policy(conv_out), self.value(conv_out)

# Creating Baseline for Policy Gradient Method
*   The Mean Buffer stores a moving average of 1 million transitions as the baseline which is subtracted from the discounted reward portion

In [ ]:
class MeanBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.deque = collections.deque(maxlen=capacity)
        self.sum = 0.0

    def add(self, val):
        if len(self.deque) == self.capacity:
            self.sum -= self.deque[0]
        self.deque.append(val)
        self.sum += val

    def mean(self):
        if not self.deque:
            return 0.0
        return self.sum / len(self.deque)

# Creating concurrent environmnets for Policy Gradient Method, A2C and A3C Method for correlated sampling problem 

In [ ]:
class MultiEnvs(gym.Wrapper):
    def __init__(self, env):
      super(MultiEnvs, self).__init__(env)
    
    def render(self, mode='human', close=True):
      pass
    
    def close(self):
      pass
    
def wrap_dqn(env):
    env = MultiEnvs(env)
    return env

In [ ]:
def make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=True, state_1d=True, random_ofs_on_reset=True,
               reward_on_close=True, volumes=False, val=False):
  if not val:
    env = StocksEnv(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL)
    env = gym.wrappers.TimeLimit(env, max_episode_steps=MAX_EPISODES)
    return wrap_dqn(env)
  else:
    env = StocksEnv(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL)
    env = gym.wrappers.TimeLimit(env, max_episode_steps=MAX_EPISODES)
    return wrap_dqn(env)

# Training the Trading Agent with the RewardTracker and TBMeanTracker Class(i.e. tracks agent status in environment)


*   These classes track the trading agent while interacting in the environment. These were taken from the ptan common library and can be found here: [common](https://github.com/Shmuma/ptan/blob/master/ptan/common/utils.py)



In [ ]:
class RewardTracker:

    # stop_reward(int): reward stopping threshold for agent, defualt is 1 
    # group_rewards(int): trading period, default is 1 trading day
    def __init__(self, writer, stop_reward=1, group_rewards=1):
        self.writer = writer
        self.stop_reward = stop_reward
        self.reward_buf = []
        self.steps_buf = []
        self.group_rewards = group_rewards

    def __enter__(self):
        self.ts = time.time()
        self.ts_frame = 0
        self.total_rewards = []
        self.done_episodes = 0
        self.total_steps = []
        return self

    def __exit__(self, *args):
        self.writer.close()

#-----------------------------Reward Section------------------------------------
    def reward(self, reward_steps, frame, epsilon=None):
        
        # Reward Per Steps
        reward, steps = reward_steps
       
        self.reward_buf.append(reward)
        self.steps_buf.append(steps)
        if len(self.reward_buf) < self.group_rewards:
            return False
        # calculates mean reward from buffer 
        reward = np.mean(self.reward_buf)
        steps = np.mean(self.steps_buf)
        self.reward_buf.clear()
        self.steps_buf.clear()

        # Total Rewards
        self.total_rewards.append(reward)

        # Total Steps
        self.total_steps.append(steps)

        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()

        # Mean Reward 
        mean_reward = np.mean(self.total_rewards[-self.group_rewards:]) 

        mean_steps = np.mean(self.total_steps[-self.group_rewards:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        print("%d: done %d games, mean reward %.3f, mean steps %.2f, speed %.2f f/s%s" % (
            frame, len(self.total_rewards)*self.group_rewards, mean_reward, mean_steps, speed, epsilon_str
        ))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("reward_per_100_tradingWindow", mean_reward, frame)
        self.writer.add_scalar("reward", reward, frame)
        self.writer.add_scalar("steps_per_100_tradingWindow", mean_steps, frame)
        if mean_reward > self.stop_reward:
            print("Kid, you're on a roll. Enjoy it while it lasts, 'cause it never does.")
            return True
        return False
    
    def reward_two(self, reward, frame, done_episodes):
      # Reward Per Steps
      self.total_rewards.append(reward)
      mean_rewards = float(np.mean(self.total_rewards[-self.group_rewards:]))
      print("%d: reward: %6.2f, mean_100: %6.2f, episodes: %d" % (frame, reward, mean_rewards, done_episodes))
      sys.stdout.flush()
      self.writer.add_scalar("reward", reward, frame)
      self.writer.add_scalar("reward_100", mean_rewards, frame)
      self.writer.add_scalar("episodes", done_episodes, frame)
      if mean_rewards > self.stop_reward:
        print("Kid, you're on a roll. Enjoy it while it lasts, 'cause it never does.")
        return True
      return False

    def reward_three(self, reward_steps, frame, epsilon=None):
        
        # Reward Per Steps
        reward = reward_steps
       
        self.reward_buf.append(reward)
        self.steps_buf.append(frame)
        if len(self.reward_buf) < self.group_rewards:
            return False
        # calculates mean reward from buffer 
        reward = np.mean(self.reward_buf)
        steps = np.mean(self.steps_buf)
        self.reward_buf.clear()
        self.steps_buf.clear()

        # Total Rewards
        self.total_rewards.append(reward)

        # Total Steps
        self.total_steps.append(steps)

        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()

        # Mean Reward 
        mean_reward = np.mean(self.total_rewards[-self.group_rewards:]) 

        mean_steps = np.mean(self.total_steps[-self.group_rewards:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        print("Step %d: Episode %d: Mean reward %.3f" % (
            frame, len(self.total_rewards)*self.group_rewards, mean_reward))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("reward_per_100_tradingWindow", mean_reward, frame)
        self.writer.add_scalar("reward", reward, frame)
        self.writer.add_scalar("steps_per_100_tradingWindow", mean_steps, frame)
        if mean_reward > self.stop_reward:
            print("Kid, you're on a roll. Enjoy it while it lasts, 'cause it never does.")
            return True
        return False


class TBMeanTracker:
    """
    TensorBoard value tracker: allows to batch fixed amount of historical values and write their mean into TB
    Designed and tested with pytorch-tensorboard in mind
    """
    def __init__(self, writer, batch_size):
        """
        :param writer: writer with close() and add_scalar() methods
        :param batch_size: integer size of batch to track
        """
        assert isinstance(batch_size, int)
        assert writer is not None
        self.writer = writer
        self.batch_size = batch_size

    def __enter__(self):
        self._batches = collections.defaultdict(list)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.writer.close()

    @staticmethod
    def _as_float(value):
        assert isinstance(value, (float, int, np.ndarray, np.generic, torch.autograd.Variable)) or torch.is_tensor(value)
        tensor_val = None
        if isinstance(value, torch.autograd.Variable):
            tensor_val = value.data
        elif torch.is_tensor(value):
            tensor_val = value

        if tensor_val is not None:
            return tensor_val.float().mean().item()
        elif isinstance(value, np.ndarray):
            return float(np.mean(value))
        else:
            return float(value)

    def track(self, param_name, value, iter_index):
        assert isinstance(param_name, str)
        assert isinstance(iter_index, int)

        data = self._batches[param_name]
        data.append(self._as_float(value))

        if len(data) >= self.batch_size:
            self.writer.add_scalar(param_name, np.mean(data), iter_index)
            data.clear()


In [ ]:
#----------------------Parameters Section---------------------------------------
# transitions to store in buffer for PGN baseline 
BASELINE_STEPS = 1000000
# num of observations for gradient update 
BATCH_SIZE = 32
BATCH_SIZE_PGN = 32
BATCH_SIZE_A2C = 32
# when to sync weights of target network for DQN
TARGET_NET_SYNC = 1000
# discount factor
GAMMA = 0.99
# determines magnitude (i.e. norm) of gradient vector
# important for varience reduction and policy changing too fast
GRAD_L2_CLIP = 0.1
# determines how many steps ahead to approximate of the total discounted 
# reward for every action
REWARD_STEPS = 2
REWARD_STEPS_PGN = 2
REWARD_STEPS_A2C = 2
# replay buffer size for DQN
REPLAY_SIZE = 100000
# intial num of samples for DQN replay buffer
REPLAY_INITIAL = 10000
# default learning rate for Adam optimizer
LEARNING_RATE = 0.001
# weight of beta parmater for entropy loss for PGN and A2C 
ENTROPY_BETA = .022
EPSILON_START = 1.0
EPSILON_STOP = 0.1
EPSILON_STEPS = 1000000
# how many concurrent envs to create
ENV_COUNT = 32
LAMBDA = 10
# when to save progress of training agent
CHECKPOINT_EVERY_STEP = 100000
# when to perform validation
VALIDATION_EVERY_STEP = 100000
CUDA = True
# date-time features, default is None
YEAR = None
STATES_TO_EVALUATE = 1000
# controls saving of model parameters
EVAL_EVERY_STEP = 1000
# max time episode default is 1000 episodes
MAX_EPISODES = 1000
# selects model type in str form can be either
# DQN, PGN, A2C, A3C, default is A3C
MODEL = "A3C"
# trading window, default is 100 days
WINDOW = 100
# reward to hit, default is infinity
STOP_REWARD = np.inf
# volume features, default is False 
VOL = False
# reset on close, default is True
RESET = True
# convolution based model, default is True
STATE1D = True
# random offset on reset, defualt is True
RANDOM = True
# reward on close, default is True (i.e. active investing)
REW_CLOSE = True
# kernel size, default is 3x3
KERNEL = 3
# number of parallel processes that are created for A3C, default is 8
PROCESSES_COUNT = 8
# number of training samples to pass to each process for processing, default is
# 32
MICRO_BATCH_SIZE = 32
# number of environments each process will use to gather data, default is 16
# which creates 16*8=128 parallel environments
NUM_ENVS = 16
REWARD_STEPS_A3C =8
#----------------------Load Data Section----------------------------------------

# dir/file of market data for training in csv form (i.e. past historical data) 
DEFAULT_STOCKS = "/content/drive/MyDrive/Datasets/SPY/spy_past.csv"

# dir/file of market data for validation in csv form (i.e. past historical data) 
DEFAULT_VAL_STOCKS = "/content/drive/MyDrive/Datasets/SPY/spy_future.csv"
SAVE_PATH = "saves"

#------------------DQN helper functions and variables Section-------------------
def calc_values_of_states(states, net, device="cpu"):
    mean_vals = []
    for batch in np.array_split(states, 64):
        states_v = torch.tensor(batch).to(device)
        action_values_v = net(states_v)
        best_action_values_v = action_values_v.max(1)[0]
        mean_vals.append(best_action_values_v.mean().item())
    return np.mean(mean_vals)


def unpack_batch_one(batch):
    states, actions, rewards, dones, last_states = [], [], [], [], []
    for exp in batch:
        state = np.array(exp.state, copy=False)
        states.append(state)
        actions.append(exp.action)
        rewards.append(exp.reward)
        dones.append(exp.last_state is None)
        if exp.last_state is None:
            last_states.append(state)       # the result will be masked anyway
        else:
            last_states.append(np.array(exp.last_state, copy=False))
    return np.array(states, copy=False), np.array(actions), np.array(rewards, dtype=np.float32), \
           np.array(dones, dtype=np.uint8), np.array(last_states, copy=False)


def calc_loss(batch, net, tgt_net, gamma, device="cpu"):
    states, actions, rewards, dones, next_states = unpack_batch_one(batch)

    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.ByteTensor(dones).to(device)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    next_state_actions = net(next_states_v).max(1)[1]
    next_state_values = tgt_net(next_states_v).gather(1, next_state_actions.unsqueeze(-1)).squeeze(-1)
    next_state_values[done_mask] = 0.0

    expected_state_action_values = next_state_values.detach() * gamma + rewards_v
    return nn.MSELoss()(state_action_values, expected_state_action_values)

#-------------------A2C helper functions and variables Section------------------
def unpack_batch(batch, net, device='cpu'):
    """
    Convert batch into training tensors
    :param batch:
    :param net:
    :return: states variable, actions tensor, reference values variable
    """
    states = []
    actions = []
    rewards = []
    not_done_idx = []
    last_states = []
    for idx, exp in enumerate(batch):
        states.append(np.array(exp.state, copy=False))
        actions.append(int(exp.action))
        rewards.append(exp.reward)
        if exp.last_state is not None:
            not_done_idx.append(idx)
            last_states.append(np.array(exp.last_state, copy=False))
    states_v = torch.FloatTensor(np.array(states, copy=False)).to(device)
    actions_t = torch.LongTensor(actions).to(device)
    # handle rewards
    rewards_np = np.array(rewards, dtype=np.float32)
    if not_done_idx:
        last_states_v = torch.FloatTensor(np.array(last_states, copy=False)).to(device)
        last_vals_v = net(last_states_v)[1]
        last_vals_np = last_vals_v.data.cpu().numpy()[:, 0]
        rewards_np[not_done_idx] += GAMMA ** REWARD_STEPS_A2C  * last_vals_np

    ref_vals_v = torch.FloatTensor(rewards_np).to(device)
    return states_v, actions_t, ref_vals_v

#-----------------A3C helper functions and variables Section--------------------

# tuple of total reward given to each  child process 
TotalReward = collections.namedtuple('TotalReward', field_names='reward')

# function that each child process will execute and deliver to the parent
def data_func(net, device, train_queue):
    envs = [make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False) for _ in range(NUM_ENVS)]
    agent = ptan.agent.PolicyAgent(lambda x: net(x)[0], device=device, apply_softmax=True)
    exp_source = ptan.experience.ExperienceSourceFirstLast(envs,
                                                           agent, 
                                                           gamma=GAMMA, 
                                                           steps_count=REWARD_STEPS)

    for exp in exp_source:
        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
            train_queue.put(TotalReward(reward=np.mean(new_rewards)))
        train_queue.put(exp)
#-----------------------------Main Section--------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if CUDA else "cpu")
    saves_path = os.path.join("/content/", SAVE_PATH)
    # want to use spawn multi-parallelism
    mp.set_start_method('spawn', force=True)
    # setting the number of threads as one, as detailed in the book, 
    # since we want to avoid multithreading issues
    os.environ['OMP_NUM_THREADS'] = "1"
    os.makedirs(saves_path, exist_ok=True)
    net = None
    exp_source = None
    # data load
    if YEAR is not None or os.path.isfile(DEFAULT_STOCKS):
        if YEAR is not None:
            stock_data = data.load_year_data(YEAR)
        else:
            stock_data = {"SPY": load_relative(DEFAULT_STOCKS)}
        
        if MODEL=="DQN":
          # training env
          env = StocksEnv(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL)
          # test env
          env_tst = StocksEnv(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL)
           # time limit 
          env = gym.wrappers.TimeLimit(env, max_episode_steps=MAX_EPISODES)
          # validation env
          val_data = {"SPY": load_relative(DEFAULT_VAL_STOCKS)}
          env_val = StocksEnv(val_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL)
        elif MODEL=="PGN":
          # training envs
          envs = [make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False) for _ in range(ENV_COUNT)]
          # test envs
          envs_tst = [make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False) for _ in range(ENV_COUNT)]
          # validation envs
          val_data = {"SPY": load_relative(DEFAULT_VAL_STOCKS)}
          envs_vals = [make_env(val_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=True) for _ in range(ENV_COUNT)]
        elif MODEL=="A2C":
           # training env
          envs = [make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False) for _ in range(ENV_COUNT)]
           # test envs
          envs_tst = [make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False) for _ in range(ENV_COUNT)]
          # validation envs
          val_data = {"SPY": load_relative(DEFAULT_VAL_STOCKS)}
          envs_vals = [make_env(val_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=True) for _ in range(ENV_COUNT)]
    else:
        raise RuntimeError("See Load Data Section above to input data for agent to train on")
    
  
    
    # tensorboard stuff
    writer = SummaryWriter(comment="-simple-" + "run")

    # model selection
    if MODEL=="DQN":
      net = DQNConv1D(env.observation_space.shape, env.action_space.n, KERNEL).to(device)
      # optimizer
      optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE) 
    elif MODEL=="PGN":
      net =  PGN(envs[0].observation_space.shape, envs[0].action_space.n, KERNEL).to(device)
      # optimizer
      optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3) 
    elif MODEL=="A2C":
      net =  A2C(envs[0].observation_space.shape, envs[0].action_space.n, KERNEL).to(device)
      optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)

#----------------------------Ptan Section---------------------------------------    
    # https://github.com/Shmuma/ptan/blob/master/ptan/agent.py
    tgt_net = ptan.agent.TargetNet(net)
    # https://github.com/Shmuma/ptan/blob/master/ptan/actions.py
    selector = ptan.actions.EpsilonGreedyActionSelector(EPSILON_START)
    # https://github.com/Shmuma/ptan/blob/master/ptan/agent.py
    if MODEL=="DQN":
      agent = ptan.agent.DQNAgent(net, selector, device=device)
      exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, GAMMA, steps_count=REWARD_STEPS_DQN)
    elif MODEL=="PGN":
      agent = ptan.agent.PolicyAgent(net,device=device,apply_softmax=True, preprocessor=ptan.agent.float32_preprocessor)
      exp_source = ptan.experience.ExperienceSourceFirstLast(envs, agent, GAMMA, steps_count=REWARD_STEPS_PGN)
    elif MODEL=="A2C":
      agent = ptan.agent.ActorCriticAgent(net,device=device,apply_softmax=True)
      exp_source = ptan.experience.ExperienceSourceFirstLast(envs, agent, GAMMA, steps_count=REWARD_STEPS_A2C)
    # https://github.com/Shmuma/ptan/blob/master/ptan/experience.py
    buffer = ptan.experience.ExperienceReplayBuffer(exp_source, REPLAY_SIZE)

#-----------------------RewardTracker/Training Loop Section---------------------
    # initialization
    step_idx = 0
    eval_states = None
    best_mean_val = None
    done_episodes = 0
    reward_sum = 0.0
    step_rewards = []
    baseline_buf = MeanBuffer(BASELINE_STEPS)

    batch_states, batch_actions, batch_scales = [], [], []

    with RewardTracker(writer, stop_reward=STOP_REWARD, group_rewards=WINDOW) as reward_tracker:
      # DQN training loop
      if MODEL=="DQN":
        while True:
          step_idx += 1
          buffer.populate(1)
          selector.epsilon = max(EPSILON_STOP, EPSILON_START - step_idx / EPSILON_STEPS)
          
          new_rewards = exp_source.pop_rewards_steps()

          if new_rewards:
            reward_tracker.reward(new_rewards[0], step_idx, selector.epsilon)
          
          if len(buffer) < REPLAY_INITIAL:
            continue
          
          # Replay Buffer
          if eval_states is None:
            print("Initial buffer populated, start training")
            eval_states = buffer.sample(STATES_TO_EVALUATE)
            eval_states = [np.array(transition.state, copy=False) for transition in eval_states]
            eval_states = np.array(eval_states, copy=False)
            
          # Training 
          if step_idx % EVAL_EVERY_STEP == 0:
            mean_val = calc_values_of_states(eval_states, net, device=device)
            writer.add_scalar("values_mean", mean_val, step_idx)
            if best_mean_val is None or best_mean_val < mean_val:
              if best_mean_val is not None:
                print("%d: Best mean value updated %.3f -> %.3f" % (step_idx, best_mean_val, mean_val))
              best_mean_val = mean_val
              torch.save(net.state_dict(), os.path.join(saves_path, "mean_val-%.3f.data" % mean_val))
              
              if step_idx >= 4000000:
                plt.clf()
                plt.plot(rewards)
                plt.title("Investing Total reward, data=%s" % "SPY ETF")
                plt.ylabel("Total Reward, %")
                plt.savefig("rewards-%s.png" % "passive investing")
                break

          optimizer.zero_grad()
          batch = buffer.sample(BATCH_SIZE)
          loss_v = calc_loss(batch, net, tgt_net.target_model, GAMMA ** REWARD_STEPS, device=device)
          loss_v.backward()
          optimizer.step()

          # Updating Target Weights
          if step_idx % TARGET_NET_SYNC == 0:
            tgt_net.sync()
          
          # Checkpointing
          if step_idx % CHECKPOINT_EVERY_STEP == 0:
            idx = step_idx // CHECKPOINT_EVERY_STEP
            torch.save(net.state_dict(), os.path.join(saves_path, "checkpoint-%3d.data" % idx))
            
          # Validation and Testing
          if step_idx % VALIDATION_EVERY_STEP == 0:
            res = validation_run(env_tst, net, device=device)
            for key, val in res.items():
              writer.add_scalar(key + "_test", val, step_idx)
            res = validation_run(env_val, net, device=device)
            for key, val in res.items():
              writer.add_scalar(key + "_val", val, step_idx)
      # PGN training loop
      elif MODEL=="PGN":
        for step_idx, exp in enumerate(exp_source):
          baseline_buf.add(exp.reward)
          baseline = baseline_buf.mean()
          batch_states.append(np.array(exp.state, copy=False))
          batch_actions.append(int(exp.action))
          batch_scales.append(exp.reward - baseline)
          
          
          # handle new rewards
          new_rewards = exp_source.pop_total_rewards()
          if new_rewards:
            done_episodes += 1
            reward = new_rewards[0]
            reward_tracker.reward_two(reward, step_idx, done_episodes)

          if len(batch_states) < BATCH_SIZE_PGN:
            continue
          
          states_v = torch.tensor(batch_states).to(device)
          batch_actions_t = torch.tensor(batch_actions).to(device)

          scale_std = np.std(batch_scales)
          batch_scale_v = torch.tensor(batch_scales).to(device)
          
          # policy loss
          optimizer.zero_grad()
          logits_v = net(states_v)
          log_prob_v = F.log_softmax(logits_v, dim=1)
          log_prob_actions_v = batch_scale_v * log_prob_v[range(BATCH_SIZE_PGN), batch_actions_t]
          loss_policy_v = -log_prob_actions_v.mean()

          # entropy loss
          prob_v = F.softmax(logits_v, dim=1)
          entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
          entropy_loss_v = -ENTROPY_BETA * entropy_v

          # calculate total loss
          loss_v = loss_policy_v + entropy_loss_v 
          loss_v.backward()
          nn_utils.clip_grad_norm_(net.parameters(), GRAD_L2_CLIP)
          optimizer.step()

          batch_states.clear()
          batch_actions.clear()
          batch_scales.clear()

          # checkpointing
          if step_idx % CHECKPOINT_EVERY_STEP == 0:
            idx = step_idx // CHECKPOINT_EVERY_STEP
            torch.save(net.state_dict(), os.path.join(saves_path, "checkpoint-%3d.data" % idx))
            
          # validation and testing
          if step_idx % VALIDATION_EVERY_STEP == 0:
            res = validation_run(envs_tst, net, device=device)
            for key, val in res.items():
              writer.add_scalar(key + "_test", val, step_idx)
            res = validation_run(envs_vals, net, device=device)
            for key, val in res.items():
              writer.add_scalar(key + "_val", val, step_idx)
      # A2C training loop
      elif MODEL=="A2C":
        with TBMeanTracker(writer, batch_size=10) as tb_tracker:
          batch = []
          for step_idx, exp in enumerate(exp_source):
            
            batch.append(exp)
            # handle new rewards
            new_rewards = exp_source.pop_total_rewards()
            if new_rewards:
              reward_tracker.reward_three(new_rewards[0], step_idx)

            if len(batch) < BATCH_SIZE_A2C:
              continue
          
            states_v, actions_t, vals_ref_v = unpack_batch(batch, net, device=device)
            batch.clear()
 
            # zero-ing gradients
            optimizer.zero_grad()
          
            # critic/value loss 
            logits_v, value_v = net(states_v)
            loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v)
          
            # actor/policy loss
            log_prob_v = F.log_softmax(logits_v, dim=1)
            adv_v = vals_ref_v - value_v.squeeze(-1).detach()
            log_prob_actions_v = adv_v * log_prob_v[range(BATCH_SIZE_A2C), actions_t]
            loss_policy_v = -log_prob_actions_v.mean()

            # entropy loss
            prob_v = F.softmax(logits_v, dim=1)
            entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
            entropy_loss_v = -ENTROPY_BETA * entropy_v
          
            # calculate policy gradient loss only
            loss_policy_v.backward(retain_graph=True)
            grads = np.concatenate([p.grad.data.cpu().numpy().flatten()
                                        for p in net.parameters()
                                        if p.grad is not None])

            # calculate entropy and value gradients
            loss_v = entropy_loss_v + loss_value_v
            loss_v.backward()
            nn_utils.clip_grad_norm_(net.parameters(), GRAD_L2_CLIP)
            optimizer.step()
          
            # get full loss
            loss_v += loss_policy_v

            # checkpointing
            if step_idx % CHECKPOINT_EVERY_STEP == 0:
              idx = step_idx // CHECKPOINT_EVERY_STEP
              torch.save(net.state_dict(), os.path.join(saves_path, "checkpoint-%3d.data" % idx))
            
            # validation and testing
            if step_idx % VALIDATION_EVERY_STEP == 0:
              res = validation_run(envs_tst, net, device=device)
              for key, val in res.items():
                writer.add_scalar(key + "_test", val, step_idx)
              res = validation_run(envs_vals, net, device=device)
              for key, val in res.items():
                writer.add_scalar(key + "_val", val, step_idx)


            tb_tracker.track("advantage",       adv_v, step_idx)
            tb_tracker.track("values",          value_v, step_idx)
            tb_tracker.track("batch_rewards",   vals_ref_v, step_idx)
            tb_tracker.track("loss_entropy",    entropy_loss_v, step_idx)
            tb_tracker.track("loss_policy",     loss_policy_v, step_idx)
            tb_tracker.track("loss_value",      loss_value_v, step_idx)
            tb_tracker.track("loss_total",      loss_v, step_idx)
            tb_tracker.track("grad_l2",         np.sqrt(np.mean(np.square(grads))), step_idx)
            tb_tracker.track("grad_max",        np.max(np.abs(grads)), step_idx)
            tb_tracker.track("grad_var",        np.var(grads), step_idx)
      # A3C training loop
      elif MODEL=="A3C":
          writer = SummaryWriter(comment="-a3c-data_")
           # training env
          env = make_env(stock_data, bars_count=BARS_COUNT, commission=DEFAULT_COMMISSION_PERC,
               reset_on_close=RESET, state_1d=STATE1D, random_ofs_on_reset=RANDOM,
               reward_on_close=REW_CLOSE, volumes=VOL, val=False)
          
          net =  A2C(env.observation_space.shape, env.action_space.n, KERNEL).to(device)
          optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)
          net.share_memory()
          
          train_queue = mp.Queue(maxsize=PROCESSES_COUNT)
          data_proc_list = []
          
          # creating the child processes
          for _ in range(PROCESSES_COUNT):
            data_proc = mp.Process(target=data_func, args=(net, device, train_queue))
            data_proc.start()
            data_proc_list.append(data_proc)

          batch = []
          step_idx = 0

          with TBMeanTracker(writer, batch_size=100) as tb_tracker:
            while True:
              train_entry = train_queue.get()
              if isinstance(train_entry, TotalReward):
                if tracker.reward(train_entry.reward, step_idx):
                  break
                continue

              step_idx += 1
              batch.append(train_entry)
              
              if len(batch) < BATCH_SIZE:
                continue

              states_v, actions_t, vals_ref_v = unpack_batch(batch, net, device=device)
              batch.clear()
 
              # zero-ing gradients
              optimizer.zero_grad()
          
              # critic/value loss 
              logits_v, value_v = net(states_v)
              loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v)
          
              # actor/policy loss
              log_prob_v = F.log_softmax(logits_v, dim=1)
              adv_v = vals_ref_v - value_v.squeeze(-1).detach()
              log_prob_actions_v = adv_v * log_prob_v[range(BATCH_SIZE_A2C), actions_t]
              loss_policy_v = -log_prob_actions_v.mean()

              # entropy loss
              prob_v = F.softmax(logits_v, dim=1)
              entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
              entropy_loss_v = -ENTROPY_BETA * entropy_v
          
              # calculate policy gradient loss only
              loss_policy_v.backward(retain_graph=True)
              grads = np.concatenate([p.grad.data.cpu().numpy().flatten()
                                        for p in net.parameters()
                                        if p.grad is not None])

              # calculate entropy and value gradients
              loss_v = entropy_loss_v + loss_value_v
              loss_v.backward()
              nn_utils.clip_grad_norm_(net.parameters(), GRAD_L2_CLIP)
              optimizer.step()
          
              # get full loss
              loss_v += loss_policy_v

              # checkpointing
              if step_idx % CHECKPOINT_EVERY_STEP == 0:
                idx = step_idx // CHECKPOINT_EVERY_STEP
                torch.save(net.state_dict(), os.path.join(saves_path, "checkpoint-%3d.data" % idx))
            
              # validation and testing
              if step_idx % VALIDATION_EVERY_STEP == 0:
                res = validation_run(envs_tst, net, device=device)
                for key, val in res.items():
                  writer.add_scalar(key + "_test", val, step_idx)
                  res = validation_run(envs_vals, net, device=device)
                for key, val in res.items():
                  writer.add_scalar(key + "_val", val, step_idx)


              tb_tracker.track("advantage",       adv_v, step_idx)
              tb_tracker.track("values",          value_v, step_idx)
              tb_tracker.track("batch_rewards",   vals_ref_v, step_idx)
              tb_tracker.track("loss_entropy",    entropy_loss_v, step_idx)
              tb_tracker.track("loss_policy",     loss_policy_v, step_idx)
              tb_tracker.track("loss_value",      loss_value_v, step_idx)
              tb_tracker.track("loss_total",      loss_v, step_idx)
          for p in data_proc_list:
            p.terminate()
            p.join()
      else:
        raise RuntimeError("Something went wrong, fix it!!!")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/

In [ ]:
!zip -r /content/runs.zip /content/runs/

In [ ]:
# !tar -xvzf  /content/drive/MyDrive/Datasets/StockMarketData/ch08-small-quotes.tgz -C /content/